## libraries

In [11]:
import requests
from bs4 import BeautifulSoup

In [12]:
class WebScraper:
    def __init__(self, url):
        self.__url = url

    @property
    def url(self):
        return self.__url
    
    @url.setter
    def url(self, url):
        self.__url = url
    
    def download_html(self, file_name='webpage.html'):
        try:
            print(f"Downloading page content from: {self.url}")
            response = requests.get(self.url)
            with open(file_name, 'w') as file:
                print(f"Saving page content to: {file_name}")
                file.write(response.text)
            print("Download completed successfully")
        except Exception as e:
            print("Unable to download page content")
            print(e)

#     Add a method extract content() to parse the HTML file using
# BeautifulSoup:
# – Extract the title of the webpage.
# – Extract all the paragraph (<p>) elements and store them in a
# list.

    def extract_content(self, file_name='webpage.html'):

        paragraphs = []
        title = None

        try:
            print(f"Extracting content from: {file_name}")
            with open(file_name, 'r') as page:
                page_soup = BeautifulSoup(page, 'html.parser')
                
                print(f"Page parsed successfully")
                
                title = page_soup.title.string
                print(f"Page title extracted: {title}")
                
                paragraphs = page_soup.find_all('p')
                print(f"Paragraphs extracted, total paragraphs: {len(paragraphs)}")
                
        except Exception as e:
            print("Unable to extract content")
            print(e)
        
        return title, paragraphs

## Testing block

In [13]:
web_scrapper = WebScraper('http://books.toscrape.com')
web_scrapper.download_html()

title, paragraphs = web_scrapper.extract_content()

print(f"Title: {title}")

for i, paragraph in enumerate(paragraphs):
    print(f"Paragraph {i+1}: {paragraph.text}")
    print()

Saving page content to: webpage.html
Download completed successfully
Extracting content from: webpage.html
Page parsed successfully
Page title extracted: 
    All products | Books to Scrape - Sandbox

Paragraphs extracted, total paragraphs: 60
Title: 
    All products | Books to Scrape - Sandbox

Paragraph 1: 







Paragraph 2: Â£51.77

Paragraph 3: 

    
        In stock
    


Paragraph 4: 







Paragraph 5: Â£53.74

Paragraph 6: 

    
        In stock
    


Paragraph 7: 







Paragraph 8: Â£50.10

Paragraph 9: 

    
        In stock
    


Paragraph 10: 







Paragraph 11: Â£47.82

Paragraph 12: 

    
        In stock
    


Paragraph 13: 







Paragraph 14: Â£54.23

Paragraph 15: 

    
        In stock
    


Paragraph 16: 







Paragraph 17: Â£22.65

Paragraph 18: 

    
        In stock
    


Paragraph 19: 







Paragraph 20: Â£33.34

Paragraph 21: 

    
        In stock
    


Paragraph 22: 







Paragraph 23: Â£17.93

Paragraph 24: 

    
        In stoc